# Bayesian

## 1. 문제
- 사람의 보편적인 인식법칙 -> ‘가장 그럴듯한’ class로 분류
- 기계(컴퓨터)의 인식 -> 수학적 틀을 이용하여 프로그래밍이 가능
- 기본적인 확률 기초이론을 사용
- P(W|X) 특징 X가 주어졌을 때 Class W 에서 발생했을 확률 (사후 확률)


# 2. 파이썬으로 텍스트 분류하기
- 온라인 전자 게시판을 위한 필터 구현
- 작성자가 부정적이거나 폭력적인 언어를 사용할 경우 필터링 (실제 필터링을 하지 않음 분류 까지만)
- 여기에서 2가지 범주로 나눔 1. 폭력적인, 0.ㅣ 폭력적이지 않은

## 2.1 텍스트로 단어 벡터 만들기
- 하나의 문장(문서)을 하나의 벡터로 변환한 단어 벡터 형태로 변환이 필요
- 1. 모든 문서에 있는 단어 고려하기 위해 중복 단어 제거 후 단어 장 생성
- 2. 각각의 문장(문서)을 벡터로 변환

# loadDataSet(): 단어 리스트와 리스트안의 문서가 폭력적인지(0) 폭력적이지 않은지(1) 분류 값을 반환
def loadDataSet():
    postingList=[['my','dog','has','flea','problems','help','please'],
                 ['maybe','not','take','him','to','dog','park','stupid'],
                ['my','dalmation','is','so','cute','I','love','him'],
                ['stop','posting','stupid','worthless','garbage'],
                ['mr','licks','ate','my','steak','how','to','stop','him'],
                ['quit','buying','wothless','dog','food','stupid']]
    classVec = [0,1,0,1,0,1] # 1: 폭력적인 0: 폭력적이지 않음
    return postingList,classVec

#createVocabList(dataset) : 단어 리스트 dataSet중 중복된 것을 제거하여 리스트로 반환
def createVocabList(dataSet):
    vocabSet = []
    for document in dataSet:
        for word in document:
            if word not in vocabSet:
                vocabSet.append(word)
    return vocabSet

#setOfWords2Vec(vocabList, inputSet): inputSet의 단어 들이, vocabList에 있으면 1, 없으면 0인 벡터 리스트 반환#
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList) 
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print "the word: %s is not in my Vocabulary" % word
    return returnVec

In [93]:
DocList, listClasses = loadDataSet()
print DocList
print listClasses


[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'wothless', 'dog', 'food', 'stupid']]
[0, 1, 0, 1, 0, 1]


### 2.1.1 모든 문서에 있는 단어 고려하기 위해 중복 단어 제거 후 단어 장 생성

In [94]:
myVocabList = createVocabList(DocList)
print myVocabList

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please', 'maybe', 'not', 'take', 'him', 'to', 'park', 'stupid', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'stop', 'posting', 'worthless', 'garbage', 'mr', 'licks', 'ate', 'steak', 'how', 'quit', 'buying', 'wothless', 'food']


### 2.1.2 각각의 문장(문서)을 벡터로 변환

In [95]:
print setOfWords2Vec(myVocabList,DocList[0])

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 2.2 단어 벡터로 확률 계산 함수 작성
- 베이지안 분류기 훈련
- 각 문장(문서)에서 출현하는 단어 count
- 전체 긍정적인 문장(문서),부정적인 문장(문서) 각각에서 해당 단어가 출현하는 확률 계산

In [96]:
from numpy import *

def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0]) ## 각 분류 항목에 대한 문서의 개수 세기
    pAbusive = sum(trainCategory) / float(numTrainDocs) #사전확률 계산 (폭력적이지 않은(1)에 대한)
    p0Num = zeros(numWords); p1Num = zeros(numWords) ##초기화
    p0Denom = 0.0; p1Denom = 0.0 
    for i in range(numTrainDocs): ## 훈련을 위한 모든 문서의 개수만큼 반복
        if trainCategory[i] ==1: ## 폭력적인 단어의 문서일 경우
            p1Num +=trainMatrix[i] ## 해당문서의 단어 갯수 증가
            p1Denom += sum(trainMatrix[i]) ## 전체 문서에서 해당 문서의 단어 갯수
        else: ## 비폭력적인 단어의 문서일 경우
            p0Num += trainMatrix[i]  ## 해당문서의 단어 갯수 증가
            p0Denom += sum(trainMatrix[i])## 전체 문서에서 해당 문서의 단어 갯수
    p1Vect = p1Num / p1Denom ## 각 폭력적인 단어들이 전체 폭력적인 단어에서 나타날 확률
    p0Vect = p0Num / p0Denom ## 각 비폭력적인 단어들이 전체 비폭력적인 단어에서 나타날 확률
        
    
    return p0Vect,p1Vect,pAbusive

In [97]:
from numpy import *

trainMat=[]
for Doc in DocList:
    trainMat.append(setOfWords2Vec(myVocabList,Doc)) ## 각 5개의 문서의 단어가 전체 단어에서의 위치 혹은 존재 여부

print trainMat

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]


In [99]:
p0V,p1V,pAb=trainNB0(trainMat,listClasses)

In [100]:
print pAb

0.5


In [101]:
for i in range(len(myVocabList)):
    print [myVocabList[i],p0V[i],p1V[i]]

['my', 0.125, 0.0]
['dog', 0.041666666666666664, 0.10526315789473684]
['has', 0.041666666666666664, 0.0]
['flea', 0.041666666666666664, 0.0]
['problems', 0.041666666666666664, 0.0]
['help', 0.041666666666666664, 0.0]
['please', 0.041666666666666664, 0.0]
['maybe', 0.0, 0.052631578947368418]
['not', 0.0, 0.052631578947368418]
['take', 0.0, 0.052631578947368418]
['him', 0.083333333333333329, 0.052631578947368418]
['to', 0.041666666666666664, 0.052631578947368418]
['park', 0.0, 0.052631578947368418]
['stupid', 0.0, 0.15789473684210525]
['dalmation', 0.041666666666666664, 0.0]
['is', 0.041666666666666664, 0.0]
['so', 0.041666666666666664, 0.0]
['cute', 0.041666666666666664, 0.0]
['I', 0.041666666666666664, 0.0]
['love', 0.041666666666666664, 0.0]
['stop', 0.041666666666666664, 0.052631578947368418]
['posting', 0.0, 0.052631578947368418]
['worthless', 0.0, 0.052631578947368418]
['garbage', 0.0, 0.052631578947368418]
['mr', 0.041666666666666664, 0.0]
['licks', 0.041666666666666664, 0.0]
['at

In [102]:
## 확률 벡터에 log를 취하도록 변경
## 분모 값을 2.0으로 초기화
## 이전 함수의 결함을 처리하기 위한 변경
## 1. 확률값이 아주 작을 경우 곱셈시 언더플로우가 일어나는것을 방지
## 2. 조건부 확률에서 0으로 나누어 지는경우 방지ㅠ

def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    p0Num=ones(numWords);p1Num=ones(numWords)  ##zeros->ones
    p0Denom=2.0;p1Denom=2.0  ##0.0->2.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    p1Vect=log(p1Num/p1Denom)      ## p1Num/p1Denom -> log(p1Num/p1Denom)
    p0Vect=log(p0Num/p0Denom)      ## p0Num/p0Denom -> log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive


In [103]:
p0V,p1V,pAb=trainNB0(trainMat,listClasses)
for i in range(len(myVocabList)):
    print [myVocabList[i],p0V[i],p1V[i]]

['my', -1.8718021769015913, -3.044522437723423]
['dog', -2.5649493574615367, -1.9459101490553135]
['has', -2.5649493574615367, -3.044522437723423]
['flea', -2.5649493574615367, -3.044522437723423]
['problems', -2.5649493574615367, -3.044522437723423]
['help', -2.5649493574615367, -3.044522437723423]
['please', -2.5649493574615367, -3.044522437723423]
['maybe', -3.2580965380214821, -2.3513752571634776]
['not', -3.2580965380214821, -2.3513752571634776]
['take', -3.2580965380214821, -2.3513752571634776]
['him', -2.1594842493533721, -2.3513752571634776]
['to', -2.5649493574615367, -2.3513752571634776]
['park', -3.2580965380214821, -2.3513752571634776]
['stupid', -3.2580965380214821, -1.6582280766035324]
['dalmation', -2.5649493574615367, -3.044522437723423]
['is', -2.5649493574615367, -3.044522437723423]
['so', -2.5649493574615367, -3.044522437723423]
['cute', -2.5649493574615367, -3.044522437723423]
['I', -2.5649493574615367, -3.044522437723423]
['love', -2.5649493574615367, -3.0445224377

## 2.3 베이지안 분류기 구현
 1. 문장(문서)에서의 모든 단어들에 대해서 부정적인 문서에서 나올 확률 계산
 2. 문장(문서)에서의 모든 단어들에 대해서 긍정적인 문서에서 나올 확률 계산
 3. 두개의 확률을 비교해서 더 높은 쪽으로 분류

In [104]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    

In [105]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat=[]

for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    
p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
testEntry = ['love', 'my', 'dalmation']
thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)
testEntry = ['stupid', 'garbage']
thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


# 3. 스팸 이메일 분류하기

## 3.1 텍스트 토큰 만들기
- 이전 절에서 다루었던 단어 벡터를 텍스트 문서로부터 어떻게 생성하는지
- 파이썬에서 스트링을 처리하는 메소드인 .split()을 사용하여 텍스트를 분할

In [106]:
MySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
print MySent.split()

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M.L.', 'I', 'have', 'ever', 'laid', 'eyes', 'upon.']


- 문제점: 구두점이 단어의 일부로 간주됨
- 해결책: 단어나 숫자에 상관없이 문장을 사용하도록 변경

In [107]:
# 구두점(.) 제거
import re
regEx = re.compile('\\W*')
listOfTokens = regEx.split(MySent)
print listOfTokens

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon', '']


## 3.2 스팸 이메일 분류 테스트

In [109]:
def textParse(bigString):    ## 큰 문자열을 단어로 쪼개어준다.
    import re
    regEx = re.compile('\\W*')
    listOfTokens = regEx.split(bigString)
    wordList = []
    
    for tok in listOfTokens:
        if (len(tok)>2):
            wordList.append(tok.lower())
            
    return wordList

wordList = textParse(open('email/spam/1.txt').read())
print wordList

['codeine', '15mg', 'for', '203', 'visa', 'only', 'codeine', 'methylmorphine', 'narcotic', 'opioid', 'pain', 'reliever', 'have', '15mg', '30mg', 'pills', '15mg', 'for', '203', '15mg', 'for', '385', '15mg', 'for', '562', 'visa', 'only']


In [114]:
docList=[]; classList = []; fullText =[]
for i in range(1,26):
    wordList = textParse(open('email/spam/%d.txt' % i).read()) ## 스팸 이메일 txt를 읽어와 단어로 분리
    docList.append(wordList) ## 단어 추가
    fullText.extend(wordList) ## 
    classList.append(1) ## classList에 1 추가 (spam문서의 클래스는 1)
    wordList = textParse(open('email/ham/%d.txt' % i).read()) ## 이메일 txt를 읽어와 단어로 분리
    docList.append(wordList) ## 단어 추가
    fullText.extend(wordList) 
    classList.append(0) ## classList에 0 추가 (이메일 문서의 클래스는 0)
    
vocabList = createVocabList(docList)# 단어 list 생성
trainingSet = range(50); testSet=[]           #훈련 집합 생성

## 10개의 난수 생성후 해당 인덱스를 테스트셋으로 분리

for i in range(10): 
    randIndex = int(random.uniform(0,len(trainingSet))) ##교차검증을 위한 난수생성
    testSet.append(trainingSet[randIndex]) ## 생성된 난수의 인덱스의 단어를 테스트 셋으로 선정
    del(trainingSet[randIndex]) ## 테스트셋이 아닌 단어들은 훈련을 위해 사용 되므로 단어 리스트에서 훈련셋 제거

trainMat=[]; trainClasses = []

for docIndex in trainingSet:  #####testNB0를 이용하여 훈련  
    trainMat.append(setOfWords2Vec(vocabList, docList[docIndex])) ## 훈련용 문서 리스트 생성
    trainClasses.append(classList[docIndex])## 훈련용 문서와 같은 위치에 클래스 배열 생성
    
p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))## 트래이닝 여기에서 pSpam은 사전확률
errorCount = 0
for docIndex in testSet:        ## 나머지 set을 이용하여 분류
    wordVector = setOfWords2Vec(vocabList, docList[docIndex])## 분류용 문서를 벡터 형태로 변환 
    if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]: ## 분류결과가 실제 결과와 같지 않은 경우
        errorCount += 1 ## 에러 추가
        print "classification error",docList[docIndex]
        
print 'the error rate is: ',float(errorCount)/len(testSet) ## 에러율 출력

classification error ['benoit', 'mandelbrot', '1924', '2010', 'benoit', 'mandelbrot', '1924', '2010', 'wilmott', 'team', 'benoit', 'mandelbrot', 'the', 'mathematician', 'the', 'father', 'fractal', 'mathematics', 'and', 'advocate', 'more', 'sophisticated', 'modelling', 'quantitative', 'finance', 'died', '14th', 'october', '2010', 'aged', 'wilmott', 'magazine', 'has', 'often', 'featured', 'mandelbrot', 'his', 'ideas', 'and', 'the', 'work', 'others', 'inspired', 'his', 'fundamental', 'insights', 'you', 'must', 'logged', 'view', 'these', 'articles', 'from', 'past', 'issues', 'wilmott', 'magazine']
classification error ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
the error rate is:  0.2
